In [1]:
%matplotlib inline
from grale.all_nb import *
import matplotlib.pyplot as plt
import numpy as np
import os
import pprint

renderers.debugDirectStderr = True
renderers.debugOutput = True

Set 'threads' as default renderer for lensplane and mass density
Set default angular unit in plotting to arcsec
Set feedback style to 'notebook'


In [2]:
class OpenCLPotentialRenderer(renderers.Renderer):
    
    def __init__(self, feedbackObject = None):
        super(OpenCLPotentialRenderer, self).__init__([ "grale_potential_opencl" ], "LENSPLANE", feedbackObject = feedbackObject)

In [6]:
Dd = 1000*DIST_MPC
Ds = 1300*DIST_MPC
Dds = 900*DIST_MPC

#l = lenses.PlummerLens(Dd, { "mass": 1e15*MASS_SUN, "width": 5*ANGLE_ARCSEC})
#l = lenses.MassSheetLens(Dd, { "Ds": Ds, "Dds": Dds })
#l = lenses.MultiplePlummerLens(Dd, [
#    { "mass": 0.7e15*MASS_SUN, "width":3*ANGLE_ARCSEC, "x": -2*ANGLE_ARCSEC, "y": 1*ANGLE_ARCSEC },
#    { "mass": 0.4e15*MASS_SUN, "width":4*ANGLE_ARCSEC, "x": 3*ANGLE_ARCSEC, "y": -1*ANGLE_ARCSEC },
#])

#l = lenses.NSIELens(Dd, { "velocityDispersion": 400000, "ellipticity": 0.8, "coreRadius": 1*ANGLE_ARCSEC})
#l = lenses.SISLens(Dd, { "velocityDispersion": 400000 })

l = lenses.CompositeLens(Dd, [
    {"lens": lenses.PlummerLens(Dd, { "mass": 0.7e15*MASS_SUN, "width":3*ANGLE_ARCSEC}),
      "x": -2*ANGLE_ARCSEC, "y": 1*ANGLE_ARCSEC, "angle": 10, "factor": 1.1},
    {"lens": lenses.PlummerLens(Dd, { "mass": 0.4e15*MASS_SUN, "width":4*ANGLE_ARCSEC}),
      "x": 3*ANGLE_ARCSEC, "y": -1*ANGLE_ARCSEC, "angle":30, "factor": 0.9}
])

In [7]:
deflScale = ANGLE_ARCSEC
potScale = ANGLE_ARCSEC*ANGLE_ARCSEC/3

intParams, floatParams = l.getCLParameters(deflScale, potScale)

print("Integers")
print(intParams)
print("Floats")
print(floatParams)

Integers
[2 2 0 2 2 0 2]
Floats
[-2.0000000e+00  1.0000000e+00  1.7453292e-01  1.1000000e+00
  5.6999888e+03  3.0000000e+00  3.0000000e+00 -1.0000000e+00
  5.2359879e-01  8.9999998e-01  3.2571362e+03  4.0000000e+00]


In [8]:
#floatBad = np.array([], dtype=np.float32)

newLens = l.createLensFromCLFloatParams(deflScale, potScale, floatParams)
print(newLens)
pprint.pprint(l.getLensParameters())
pprint.pprint(newLens.getLensParameters())

[{'angle': 10.0,
  'factor': 1.1,
  'lens': <grale.lenses.PlummerLens object at 0x7f82d6a1a8d0>,
  'x': -9.69627362219072e-06,
  'y': 4.84813681109536e-06},
 {'angle': 30.0,
  'factor': 0.9,
  'lens': <grale.lenses.PlummerLens object at 0x7f82d6a19350>,
  'x': 1.454441043328608e-05,
  'y': -4.84813681109536e-06}]
[{'angle': 9.99999970909292,
  'factor': 1.100000023841858,
  'lens': <grale.lenses.PlummerLens object at 0x7f82d6a1a8d0>,
  'x': -9.69627362219072e-06,
  'y': 4.84813681109536e-06},
 {'angle': 30.000000834826057,
  'factor': 0.8999999761581421,
  'lens': <grale.lenses.PlummerLens object at 0x7f82d6a1aaf0>,
  'x': 1.454441043328608e-05,
  'y': -4.84813681109536e-06}]


In [9]:
#pprint.pprint(l.getLensParameters()[0]["lens"].getLensParameters())
#pprint.pprint(newLens.getLensParameters()[0]["lens"].getLensParameters())

In [10]:
#pprint.pprint(l.getLensParameters()[1]["lens"].getLensParameters())
#pprint.pprint(newLens.getLensParameters()[1]["lens"].getLensParameters())

In [11]:
l.getCLAdjustableFloatingPointParameterInfo()

[{'name': 'x_0_scaled', 'offset': 0, 'hard_min': -inf, 'hard_max': inf},
 {'name': 'y_0_scaled', 'offset': 1, 'hard_min': -inf, 'hard_max': inf},
 {'name': 'angle_0', 'offset': 2, 'hard_min': -inf, 'hard_max': inf},
 {'name': 'factor_0', 'offset': 3, 'hard_min': 0.0, 'hard_max': inf},
 {'name': 'lens_0,mass_scaled', 'offset': 4, 'hard_min': 0.0, 'hard_max': inf},
 {'name': 'lens_0,width_scaled',
  'offset': 5,
  'hard_min': 0.0,
  'hard_max': inf},
 {'name': 'x_1_scaled', 'offset': 6, 'hard_min': -inf, 'hard_max': inf},
 {'name': 'y_1_scaled', 'offset': 7, 'hard_min': -inf, 'hard_max': inf},
 {'name': 'angle_1', 'offset': 8, 'hard_min': -inf, 'hard_max': inf},
 {'name': 'factor_1', 'offset': 9, 'hard_min': 0.0, 'hard_max': inf},
 {'name': 'lens_1,mass_scaled',
  'offset': 10,
  'hard_min': 0.0,
  'hard_max': inf},
 {'name': 'lens_1,width_scaled',
  'offset': 11,
  'hard_min': 0.0,
  'hard_max': inf}]